# 1. Determinación de bases y promedios para tramos

## A. Info a utilizar

* CWTR SUM 
* XML (otro empleador)
* Medidas SAP (IT 0)

## B. Importes consolidados por mes

In [1]:
import pandas as pd
from data import DOTACION, CUILES, F572_WEB
from deducciones.incremental.calculo import *


from helpers.dict import unpack
from helpers.progressbar import ProgressBar

In [2]:
#primero tengo que consolidar la info que traigo de SAP (CWTR_SUM) y el XML (ingresos otro empleador)

#me traigo la info de la CWTR_SUM ya que esta los tiene agrupados por importe por mes
df_rem_mes = []

legajos_dotacion = list(DOTACION['legajo'])

p = ProgressBar(len(legajos_dotacion) * 12)
for legajo in legajos_dotacion:

    for m in range(1,13):
        restar_prorrateos = DeduccionEspecial.obtener_rem_mes(legajo,m)
        df_rem_mes.append({'legajo': legajo, 'mes': m, 'importe': restar_prorrateos})
        p.next()

df_rem_mes = pd.DataFrame.from_records(df_rem_mes)
df_rem_mes

|██████████████████████████████████████████████████| 100.0 %


,legajo,mes,importe
0,6000002,1,466593.92
1,6000002,2,1405581.67
2,6000002,3,359751.07
3,6000002,4,363686.19
4,6000002,5,363361.69
...,...,...,...
37063,6109479,8,0.00
37064,6109479,9,0.00
37065,6109479,10,168264.70
37066,6109479,11,264920.89


In [3]:
# dejo las columnas que necesito
df_filtrado = F572_WEB.query('codigo == "Q-OE-ganBrut"')
df_filtrado = df_filtrado[['cuil', 'mes', 'importe']].reset_index(drop=True)

#paso la columna cuil a int, ya que si no el cruce me da error
df_filtrado["cuil"]=df_filtrado["cuil"].astype("int64")

#agrupo por importe los ingresos de otro empleador
df_base_OE = df_filtrado.groupby(['cuil','mes'])['importe'].sum().reset_index() 
df_base_OE

,cuil,mes,importe
0,20201963096,1,16296.94
1,20201963096,2,17215.09
2,20201963096,3,19842.10
3,20201963096,4,19842.10
4,20201963096,5,19842.10
...,...,...,...
58,27279548375,8,17043.89
59,27279548375,9,17813.89
60,27279548375,10,17923.89
61,27279548375,11,18690.77


In [4]:
#como solo tengo el dato del cuil, curzo con el archivo CUILES y me traigo el legajo
df_oe_unido = df_base_OE.merge(CUILES, how='left', on='cuil')

print("[!] Se eliminarán siguientes datos por no ser un CUIL a procesar:")
display(df_oe_unido[df_oe_unido['legajo'].isnull()])

df_oe_unido = df_oe_unido[['legajo', 'mes', 'importe']].reset_index(drop=True)

#luego de que ya tengo el legajo, concateno con el dataframe de la cwtr y ahora tengo el total de rem por mes
df_rem_mes_finales = pd.concat([df_rem_mes, df_oe_unido], ignore_index=True)

# Se eliminan aquellos registros de empleados que no están en el listado de cuits:
df_rem_mes_finales.dropna(inplace=True)

[!] Se eliminarán siguientes datos por no ser un CUIL a procesar:


,cuil,mes,importe,legajo,nombre


In [5]:
# Se unen todos los datos en 1
df_rem_mes_finales = df_rem_mes_finales.groupby(['legajo','mes'])['importe'].sum().reset_index() 

## C. Cálculo de promedios a cada mes 

In [6]:
df_promedios = []
p = ProgressBar(len(df_rem_mes_finales))

for index, row in df_rem_mes_finales.iterrows():
    legajo, mes, importe = unpack(row, 'legajo', 'mes', 'importe')
    
    norma = DeduccionEspecial.norma_aplicable(mes)
    valor_promedio = DeduccionEspecial.obtener_promedio_rem_df(legajo, mes, norma, df_rem_mes_finales)
    
    base_tramo = min(importe,valor_promedio)
    tramo = DeduccionEspecial.obtener_tramo(base_tramo, mes)

    df_promedios.append({
        'legajo':       legajo,
        'mes':          mes,
        'importe':      importe,
        'promedio':     valor_promedio,
        'base_tramo':   base_tramo,
        'tramo':        tramo,
    })
    p.next()

df_promedios = pd.DataFrame.from_records(df_promedios)
df_promedios

|██████████████████████████████████████████████████| 100.0 %


,legajo,mes,importe,promedio,base_tramo,tramo
0,6000002.0,1.0,466593.92,466593.92,466593.92,3
1,6000002.0,2.0,1405581.67,936087.80,936087.80,3
2,6000002.0,3.0,359751.07,743975.55,359751.07,3
3,6000002.0,4.0,363686.19,648903.21,363686.19,3
4,6000002.0,5.0,363361.69,591794.91,363361.69,3
...,...,...,...,...,...,...
37063,6114237.0,8.0,0.00,0.00,0.00,1
37064,6114237.0,9.0,0.00,0.00,0.00,1
37065,6114237.0,10.0,0.00,0.00,0.00,1
37066,6114237.0,11.0,0.00,0.00,0.00,1


In [9]:
#lo guardo en un Excel
df_promedios.to_excel("../bases-ganancias-2022/info/bases_por_mes.xlsx")

In [8]:
#se deja la base guardada en middle para despues ser utilizada en otros cálculos 
df_promedios.to_parquet('../bases-ganancias-2022/middle/bases_tramos.parquet', compression='brotli')